<a href="https://colab.research.google.com/github/isiger/wqu_msc_fe/blob/main/Group_5_G_Notebook_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Group Work Submission II

## Environment setup

In [ ]:
#!/usr/bin/env python
# coding: utf-8

In [ ]:
!pip install yfinance
!pip install pmdarima
!pip install arch
!pip install statsmodels
!pip install pyflux

     |████████████████████████████████| 5.5MB 6.6MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.55-py2.py3-none-any.whl size=22616 sha256=f5850039f83f80071a6cdf477f2d7eb8de8a167d5934023f0291ae9813b345ae
  Stored in directory: /root/.cache/pip/wheels/04/98/cc/2702a4242d60bdc14f48b4557c427ded1fe92aedf257d4565c
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
     |████████████████████████████████| 1.5MB 4.3MB/s 
     |████████████████████████████████| 9.5MB 16.9MB/s 
     |████████████████████████████████| 2.1MB 43.3MB/s 
  Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2
  Found existing installation: Cython 0.29.21
    Uninstalling Cython-0.29.21:
      Successfully uninstalled Cython-0.29.21
     |████████████████████████████████| 798kB 4.2MB/s 
     |████████████████████████████████| 1.3MB 4.1MB/s 

### Imports

In [ ]:
import time
# Data Gathering
import yfinance as yf
import requests
import urllib.request
# Data Processing
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from pandas import datetime
# Data Visualization
import matplotlib.pyplot as plt
# ARCH and GARCH model
import pyflux as pf
from arch import arch_model
from arch.univariate import ConstantMean, GARCH, EGARCH, Normal,FIGARCH
# Checking for stationarity
from statsmodels.tsa.stattools import adfuller
# Engle-Granger Cointegration
from statsmodels.tsa.stattools import coint
from statsmodels.tsa.vector_ar.vecm import VECM, select_order, coint_johansen

from IPython.display import set_matplotlib_formats
set_matplotlib_formats('pdf', 'svg')

%matplotlib inline

### 0.1 Gold Exchange Traded Fund.
*SPDR Gold MiniShares* (**GLDM**)

In [ ]:
goldFund = 'GLDM'

### 0.2 Equity ETF from outside the US
*Invesco India ETF* (**PIN**)

In [ ]:
equityETF = 'PIN'

### 0.3 Bitcoin

In [ ]:
bitcoin = 'BTC-USD'

## 1 Data Importing

### 1.0 Import closing prices for the entire year of 2020 for  your GOLD ETF, your equity ETF, and bitcoin

### 1.1 GLDM gold ETF daily closing prices from Apr to Dec 2020



In [ ]:
tickerSymbol = goldFund
tickerData = yf.Ticker(tickerSymbol)
#historical prices for this ticker
goldDf = tickerData.history(period='1d', start='2020-04-01', end='2020-12-31')
goldDf_Q2 = tickerData.history(period='1d', start='2020-04-01', end='2020-06-30') # Q2
goldDf_Q3 = tickerData.history(period='1d', start='2020-07-01', end='2020-09-30') # Q3
goldDf_Q4 = tickerData.history(period='1d', start='2020-10-01', end='2020-12-31') # Q4
goldextraDf = tickerData.history(period='1d', start='2020-03-01', end='2020-12-31') # March month for moving average
goldDf

### 1.2 PIN equity ETF daily closing prices for Apr to Dec 2020

In [ ]:
tickerSymbol = equityETF
tickerData = yf.Ticker(equityETF)
idx = pd.date_range('2020-04-01', '2020-12-31')
equityDf = tickerData.history(period='1d', start='2020-04-01', end='2020-12-31')
equityDf_Q2 = tickerData.history(period='1d', start='2020-04-01', end='2020-06-30') # Q2
equityDf_Q3 = tickerData.history(period='1d', start='2020-07-01', end='2020-09-30') # Q3
equityDf_Q4 = tickerData.history(period='1d', start='2020-10-01', end='2020-12-31') # Q4
equityextraDf = tickerData.history(period='1d', start='2020-03-01', end='2020-12-31') # March month for moving avg
equityDf

### 1.3 Bitcoin daily closing prices from Apr to Dec 2020

In [ ]:
tickerSymbol = bitcoin
tickerData = yf.Ticker(bitcoin)
bitcoinDf = tickerData.history(period='1d', start='2020-04-01', end='2020-12-31').reindex(index=goldDf.index)
bitcoinDf_Q2 = tickerData.history(period='1d', start='2020-04-01', end='2020-06-30').reindex(index=goldDf_Q2.index) # Q2
bitcoinDf_Q3 = tickerData.history(period='1d', start='2020-07-01', end='2020-09-30').reindex(index=goldDf_Q3.index) # Q3
bitcoinDf_Q4 = tickerData.history(period='1d', start='2020-10-01', end='2020-12-31').reindex(index=goldDf_Q4.index) # Q4
bitcoinextraDf = tickerData.history(period='1d', start='2020-03-01', end='2020-12-31').reindex(index=goldextraDf.index) # March month for moving avg
bitcoinDf

## 2 Data Processing

### 2.1.1 Daily returns of Gold ETF

In [ ]:
GoldDailyReturnsDf = goldDf['Close'].pct_change()
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_axes([0.1,0.1,2,2])
ax1.plot(GoldDailyReturnsDf)
ax1.set_xlabel("Date")
ax1.set_ylabel("Percent")
ax1.set_title("Gold Daily returns data") 
plt.show()
GoldDailyReturnsDf.dropna()

### 2.1.2 Daily returns of Equity ETF

In [ ]:
EquityDailyReturnsDf = equityDf['Close'].pct_change()
fig = plt.figure()
ax1 = fig.add_axes([0.1,0.1,2,2])
ax1.plot(EquityDailyReturnsDf)
ax1.set_xlabel("Date")
ax1.set_ylabel("Percent")
ax1.set_title("Equity Daily returns data")
plt.show()
EquityDailyReturnsDf.dropna()

### 2.1.3 Daily returns of Bitcoin

In [ ]:
BitcoinDailyReturnsDf = bitcoinDf['Close'].pct_change()
fig = plt.figure()
ax1 = fig.add_axes([0.1,0.1,2,2])
ax1.plot(BitcoinDailyReturnsDf)
ax1.set_xlabel("Date")
ax1.set_ylabel("Percent")
ax1.set_title("Bitcoin Daily returns data")
plt.show()
BitcoinDailyReturnsDf.dropna()

## 3 Data Summary 


### 3.1 Compute a 20-day moving average price of your GOLD ETF

In [ ]:
goldCloseDf = goldextraDf['Close']
goldDf['SMA_20'] = goldCloseDf.rolling(20, min_periods=1).mean()
goldDf['SMA_20']

### 3.2 Compute a 20-day moving average price of your EQUITY ETF

In [ ]:
equityCloseDf = equityextraDf['Close']
equityDf['SMA_20'] = equityCloseDf.rolling(20, min_periods=1).mean()
equityDf['SMA_20']

### Compute a 20-day moving average price of BITCOIN

In [ ]:
bitcoinCloseDf = bitcoinextraDf['Close']
bitcoinDf['SMA_20'] = bitcoinCloseDf.rolling(20, min_periods=1).mean()
bitcoinDf['SMA_20']

##4 Graphing



## 4.1.1 Gold ETF price vs 20-DAY Moving Average

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(goldDf['Close'],label = "Close Price")
plt.plot(goldDf['SMA_20'],label = "20-SMA")
plt.xlabel('Time')  
plt.ylabel('Prices')  
plt.title('Gold ETF - Prices VS SMA_20') 
plt.legend() 
plt.show()

### 4.1.2 Equity ETF price vs 20-DAY Moving Average

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(equityDf['Close'],label = "Close Price")
plt.plot(equityDf['SMA_20'],label = "20-SMA")
plt.xlabel('Time')  
plt.ylabel('Prices')  
plt.title('Equity ETF - Prices VS SMA_20') 
plt.legend() 
plt.show()

### 4.1.3 Bitcoin price vs 20-DAY Moving Average

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(bitcoinDf['Close'],label = "Close Price")
plt.plot(bitcoinDf['SMA_20'],label = "20-SMA")
plt.xlabel('Time')  
plt.ylabel('Prices')  
plt.title('Bitcoin ETF - Prices VS SMA_20') 
plt.legend() 
plt.show()

### 4.2 how the price series and average series intersect or not.

### 4.3 Gold vs Equity prices

In [ ]:

fig, ax1 = plt.subplots(figsize=(12,8))

color = 'tab:red'
ax1.set_xlabel('Time')
ax1.set_ylabel('Gold Prices', color=color)
ax1.plot( goldDf['Close'], color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:blue'
ax2.set_ylabel('Equity Prices', color=color)  # we already handled the x-label with ax1
ax2.plot(equityDf['Close'], color=color)
ax2.tick_params(axis='y', labelcolor=color)

plt.show()

### 4.4 Gold vs Bitcoin Graphs

In [ ]:
fig, ax1 = plt.subplots(figsize=(12,8))

color = 'tab:red'
ax1.set_xlabel('Time')
ax1.set_ylabel('Gold Prices', color=color)
ax1.plot( goldDf['Close'], color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:blue'
ax2.set_ylabel('Bitcoin Prices', color=color)  # we already handled the x-label with ax1
ax2.plot(bitcoinDf['Close'], color=color)
ax2.tick_params(axis='y', labelcolor=color)

plt.show()

### 4.5 Equity vs Bitcoin Graphs


In [ ]:
fig, ax1 = plt.subplots(figsize=(12,8))

color = 'tab:red'
ax1.set_xlabel('Time')
ax1.set_ylabel('Equity Prices', color=color)
ax1.plot( equityDf['Close'], color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:blue'
ax2.set_ylabel('Bitcoin Prices', color=color)  # we already handled the x-label with ax1
ax2.plot(bitcoinDf['Close'], color=color)
ax2.tick_params(axis='y', labelcolor=color)

plt.show()

## 5 Fitting a GARCH Model

##5.1 Fitting GARCH(1,1) model on April-December data of GLDM


In [ ]:
GoldDailyReturnsDf = goldDf['Close'].pct_change().dropna()
train = GoldDailyReturnsDf[:165]
test = GoldDailyReturnsDf[-25:]
scaling_factor = 10 
model = ConstantMean(train*scaling_factor)
model.volatility = GARCH(1, 0, 1)
model.distribution = Normal()
# fit model
model_fit = model.fit()
model_fit.summary()

In [ ]:
# forecast the test set
prediction = pd.DataFrame(model_fit.forecast(horizon=25).variance.values[-1, :]/scaling_factor, index=test.index)
prediction.columns = ['predicted_price']
prediction

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(train,label="Training")
plt.plot(test,label="Test")
plt.plot(prediction,label="Predicted")
plt.xlabel('Time')
plt.ylabel('Returns')
plt.legend()
plt.show()

##5.2 Fitting GARCH-M model on April-December data of GLDM


In [ ]:
GoldReturnsDf = goldDf['Close'].pct_change().dropna()
train = GoldReturnsDf[:200]
test = GoldReturnsDf[-5:]
#scaling_factor = 100 
# fit model
model = arch_model(GoldReturnsDf, mean='AR',lags=1)
model_fit = model.fit(update_freq=5)
model_fit.summary() 

In [ ]:
# forecast the test set
prediction = pd.DataFrame(model_fit.forecast(horizon=5).variance.values[-1, :]*100, index=test.index)
prediction.columns = ['predicted_price']
prediction

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(train,label="Training")
plt.plot(test,label="Test")
plt.plot(prediction,label="Predicted")
plt.xlabel('Time')
plt.ylabel('Returns')
plt.legend()
plt.show()

##5.3 Fitting E-GARCH, T-GARCH, and I-GARCH model on April-December data of GLDM

###5.3.1 Fitting E-GARCH model on April-December data of GLDM


In [ ]:
GoldDailyReturnsDf = goldDf['Close'].pct_change().dropna()
train = GoldDailyReturnsDf[-75:]
test = GoldDailyReturnsDf[-1:]
scaling_factor = 100
model = ConstantMean(train*scaling_factor)
model.volatility = EGARCH(1, 1, 1)
model.distribution = Normal()
# fit model
model_fit = model.fit()
model_fit.summary()

In [ ]:
# forecast the test set
prediction = pd.DataFrame(model_fit.forecast(horizon=1).variance.values[-1, :]/scaling_factor, index=test.index)
prediction.columns = ['predicted_price']
prediction

###5.3.2 Fitting T-GARCH model on April-December data of GLDM


In [ ]:
GoldDailyReturnsDf = goldDf['Close'].pct_change().dropna()
train = GoldDailyReturnsDf[-75:]
test = GoldDailyReturnsDf[-1:]
model = arch_model(GoldDailyReturnsDf, vol='GARCH', power=2.0, p=1, o=1, q=1)
model_fit = model.fit(update_freq=5)
model_fit.summary()

In [ ]:
# forecast the test set
prediction = pd.DataFrame(model_fit.forecast(horizon=1).variance.values[-1, :]*100, index=test.index)
prediction.columns = ['predicted_price']
prediction

###5.3.3 Fitting I-GARCH model on April-December data of GLDM


In [ ]:
GoldDailyReturnsDf = goldDf['Close'].pct_change().dropna()
train = GoldDailyReturnsDf[-75:]
test = GoldDailyReturnsDf[-1:]
scaling_factor = 100
model = ConstantMean(train*scaling_factor)
model.volatility = FIGARCH(1, 1, 1)
model.distribution = Normal()
# fit model
model_fit = model.fit()
model_fit.summary()

In [ ]:
# forecast the test set
prediction = pd.DataFrame(model_fit.forecast(horizon=1).variance.values[-1, :]/scaling_factor, index=test.index)
prediction.columns = ['predicted_price']
prediction

##5.4 Long Term Variances of 5 MODELS

###5.4.1 Long Run Variance of **GARCH** MODEL

In [ ]:
#long_run_variance = omega/(1-alpha-beta)
long_run_variance = 0.0140

###5.4.2 Long Run Variance of **GARCH-M** MODEL

In [ ]:
#long_run_variance = omega/(1-alpha-beta)
long_run_variance = 0.1116

###5.4.3 Long Run Variance of **E-GARCH** MODEL

In [ ]:
#long_run_variance = e^(omega/(1-beta))
long_run_variance = 0.6151

###5.4.4 Long Run Variance of **T-GARCH** MODEL

In [ ]:
#long_run_variance = e^(omega/(1-alpha-beta))
long_run_variance = 1

###5.4.5 Long Run Variance of **I-GARCH** MODEL

In [ ]:
#long_run_variance = omega/(1-beta)
long_run_variance = 0.66

# 6 Assessing stationarity

### The Augmented Dickey-Fuller test is a type of statistical test called a unit root test, which is used to check stationarity in data using 
A p-value below a threshold (such as 5% or 1%) suggests we reject the null hypothesis (stationary), otherwise a p-value above the threshold suggests we fail to reject the null hypothesis (non-stationary).

- p-value > 0.05: Fail to reject the null hypothesis (H0), the data has a unit root and is non-stationary.
- p-value <= 0.05: Reject the null hypothesis (H0), the data does not have a unit root and is stationary. 

### 6.1 Gold ETF April-December Stationary Test

In [ ]:
result = adfuller(goldDf['Close'])
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

Test used is Augmented Dickey-Fuller test --> It is non-stationary

### 6.2 Equity ETF April-December Stationary Test

In [ ]:
result = adfuller(equityDf['Close'])
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

Test used is Augmented Dickey-Fuller test --> It is non-stationary

### 6.3 Bitcoin April-December Stationary Test

In [ ]:
result = adfuller(bitcoinDf['Close'])
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

Test used is Augmented Dickey-Fuller test --> It is non-stationary

# 7 Modelling Cointegration

### 7.1 Engle-Granger Cointegration Test for Q2

Equity vs Gold

In [ ]:
coint(equityDf_Q2['Close'], goldDf_Q2['Close'])

In [ ]:
coint(equityDf_Q2['Close'], bitcoinDf_Q2['Close'])

In [ ]:
coint(goldDf_Q2['Close'], bitcoinDf_Q2['Close'])

### 7.2 Presence of co-integrating vectors


### 7.3 Vector Error Correction model

In [ ]:

df_Q2 = pd.DataFrame(index=goldDf_Q2.index, data={'gold': goldDf_Q2['Close'], 'equity': equityDf_Q2['Close'], 'bitcoin':bitcoinDf_Q2['Close']} )

model = VECM(df_Q2, coint_rank=1)


### 7.7 Predicting Q3 cointegration from Q2 cointegration

### 7.8 

In [ ]:
coint_johansen()